In [ ]:
# pip install bs4
# pip install selenium
# pip install requests
# pip install webdriver-manager

https://medium.com/analytics-vidhya/web-scraping-google-search-results-with-selenium-and-beautifulsoup-4c534817ad88

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
# Import the beautifulsoup and request libraries of python.
import requests
import bs4

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

/var/folders/m5/tw1ymsrj4jjg5vcyxyqy8r280000gn/T/ipykernel_38649/1014324415.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
/var/folders/m5/tw1ymsrj4jjg5vcyxyqy8r280000gn/T/ipykernel_38649/1014324415.py:3: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)


In [24]:
from ast import Num
from magic_google import MagicGoogle
import pprint

PROXIES = None
# PROXIES = [{
#     'http': 'http://192.168.2.207:1080',
#     'https': 'http://192.168.2.207:1080'
# }]

# Or MagicGoogle()
mg = MagicGoogle(PROXIES)
links=[]
# Crawling url
for url in mg.search_url(query='site:upenn.edu "innovation"', num=1000):
    links.append(url)
    

2022-10-19 10:47:25,014 - magic_google - INFO: https://www.google.com/search?q=site%3Aupenn.edu+%22innovation%22&btnG=Search&gbv=1&num=1000


In [83]:
import pandas as pd
ECOSYSTEM_PENN_LIST = [
'https://www.sas.upenn.edu/',
'https://venturelab.upenn.edu/',
'https://www.asc.upenn.edu/',
'https://www.dental.upenn.edu/',
'https://www.design.upenn.edu/',
'https://www.gse.upenn.edu/',
'https://www.seas.upenn.edu/',
'https://www.law.upenn.edu/',
'https://www.med.upenn.edu/',
'https://www.nursing.upenn.edu/',
'https://www.sp2.upenn.edu/',
'https://www.vet.upenn.edu/',
'https://pci.upenn.edu/',
'https://www.library.upenn.edu/',
'https://www.chop.edu/',
'https://sciencecenter.org/',
'https://wistar.org/']
df5 = pd.DataFrame(ECOSYSTEM_PENN_LIST)

In [84]:
import pandas as pd
df1 = pd.read_csv('GoogleSearch/siteupenn.edu innovation (1).csv',delimiter=';')
df2 = pd.read_csv('GoogleSearch/siteupenn.edu innovation (2).csv',delimiter=';')
df3 = pd.read_csv('GoogleSearch/siteupenn.edu innovation (3).csv',delimiter=';')
df4 = pd.read_csv('GoogleSearch/siteupenn.edu innovation (4).csv',delimiter=';')
df1 = df1.Url
df2 = df2.Url
df3 = df3.Url
df4 = df4.Url

In [93]:
from dataclasses import replace
df = pd.concat([df1,df2,df3,df4,df5])
df = df.reset_index().drop(columns=['index'])
df.drop_duplicates(inplace=True)
df.rename(columns={0:'url'}, inplace=True)
df.to_csv('total_links.csv')

In [96]:
import pandas as pd
links = pd.read_csv('total_links.csv', index_col=0)

In [97]:
links.head()

,url
0,https://www.nursing.upenn.edu/innovation/
1,https://www.nursing.upenn.edu/innovation/innov...
2,https://knowledge.wharton.upenn.edu/article/bu...
3,https://knowledge.wharton.upenn.edu/category/i...
4,https://www.med.upenn.edu/evpdean/innovation.html


In [99]:
sum(links.url.duplicated()) # check for duplication

0

In [100]:
len(links)

404

In [102]:
domains = []
for link in links.url:
    link_element = link.split('/')
    domains.append(link_element[2].replace("www.", ""))

In [103]:
links['domain'] = domains

In [104]:
len(list(set(domains)))

76

In [105]:
links

,url,domain
0,https://www.nursing.upenn.edu/innovation/,nursing.upenn.edu
1,https://www.nursing.upenn.edu/innovation/innov...,nursing.upenn.edu
2,https://knowledge.wharton.upenn.edu/article/bu...,knowledge.wharton.upenn.edu
3,https://knowledge.wharton.upenn.edu/category/i...,knowledge.wharton.upenn.edu
4,https://www.med.upenn.edu/evpdean/innovation.html,med.upenn.edu
...,...,...
399,https://www.vet.upenn.edu/,vet.upenn.edu
401,https://www.library.upenn.edu/,library.upenn.edu
402,https://www.chop.edu/,chop.edu
403,https://sciencecenter.org/,sciencecenter.org


In [107]:
import requests
innovate_count = []
for i in links.url:
    page = requests.get(i, verify=False)
    count = sum(page.text.count(x) for x in ('innovation', 'innovations', 'innovate', 'innovative'))
    innovate_count.append(count)
innovate_count

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nursing.upenn.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nursing.upenn.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'knowledge.wharton.upenn.edu'. Adding certificate verification is strongly advised. See: https://urllib3.read

[52,
 49,
 89,
 36,
 6,
 87,
 40,
 19,
 1,
 6,
 0,
 82,
 7,
 117,
 141,
 4,
 7,
 81,
 43,
 30,
 48,
 15,
 69,
 11,
 57,
 27,
 0,
 119,
 5,
 59,
 45,
 16,
 43,
 3,
 0,
 7,
 95,
 13,
 4,
 31,
 149,
 34,
 10,
 5,
 5,
 26,
 36,
 87,
 40,
 27,
 0,
 9,
 31,
 13,
 32,
 7,
 71,
 68,
 15,
 27,
 14,
 14,
 10,
 14,
 42,
 5,
 13,
 13,
 28,
 4,
 46,
 17,
 2,
 97,
 41,
 5,
 0,
 104,
 7,
 3,
 10,
 27,
 4,
 0,
 26,
 20,
 20,
 19,
 0,
 1,
 0,
 5,
 3,
 29,
 14,
 42,
 18,
 25,
 28,
 77,
 5,
 17,
 5,
 19,
 1,
 21,
 34,
 34,
 148,
 53,
 13,
 25,
 156,
 9,
 0,
 0,
 25,
 26,
 8,
 0,
 17,
 5,
 6,
 19,
 8,
 22,
 12,
 15,
 21,
 29,
 19,
 55,
 50,
 7,
 10,
 0,
 5,
 0,
 53,
 0,
 0,
 2,
 56,
 29,
 77,
 11,
 35,
 21,
 12,
 73,
 2,
 28,
 48,
 4,
 25,
 21,
 19,
 3,
 24,
 6,
 20,
 3,
 0,
 11,
 19,
 48,
 19,
 0,
 103,
 11,
 36,
 3,
 10,
 4,
 4,
 55,
 44,
 0,
 85,
 0,
 23,
 26,
 1,
 50,
 5,
 1,
 34,
 19,
 2,
 17,
 15,
 85,
 41,
 10,
 29,
 48,
 20,
 81,
 18,
 23,
 24,
 25,
 20,
 23,
 69,
 12,
 53,
 17,
 82,
 31,
 40,
 13

In [108]:
links['innovate_count'] = innovate_count

In [138]:
links

,url,domain,innovate_count
0,https://www.nursing.upenn.edu/innovation/,nursing.upenn.edu,52
1,https://www.nursing.upenn.edu/innovation/innov...,nursing.upenn.edu,49
2,https://knowledge.wharton.upenn.edu/article/bu...,knowledge.wharton.upenn.edu,89
3,https://knowledge.wharton.upenn.edu/category/i...,knowledge.wharton.upenn.edu,36
4,https://www.med.upenn.edu/evpdean/innovation.html,med.upenn.edu,6
...,...,...,...
399,https://www.vet.upenn.edu/,vet.upenn.edu,0
401,https://www.library.upenn.edu/,library.upenn.edu,1
402,https://www.chop.edu/,chop.edu,1
403,https://sciencecenter.org/,sciencecenter.org,4


Collapsed spreadsheet: domains, Col: how many pages are there, and how many times innovation is mentioned

In [116]:
links.groupby('domain')['url'].count()

domain
accelerationlab.upenn.edu           1
aisp.upenn.edu                      2
almanac.upenn.edu                   3
apps.wharton.upenn.edu              2
asc.upenn.edu                       1
                                   ..
wistar.org                          1
wrds-wharton.upenn.edu              1
wsp.wharton.upenn.edu               2
www-management.wharton.upenn.edu    1
yourimpact.upenn.edu                1
Name: url, Length: 76, dtype: int64

domain
accelerationlab.upenn.edu           45
aisp.upenn.edu                       0
almanac.upenn.edu                    4
apps.wharton.upenn.edu               0
asc.upenn.edu                        0
                                    ..
wistar.org                           0
wrds-wharton.upenn.edu              21
wsp.wharton.upenn.edu                0
www-management.wharton.upenn.edu     0
yourimpact.upenn.edu                18
Name: innovate_count, Length: 76, dtype: int64

In [140]:
groupby_domain = pd.DataFrame(links.groupby('domain')['url'].count())

In [141]:
groupby_domain['innovate_count'] = links.groupby('domain')['innovate_count'].sum()

In [142]:
groupby_domain.rename(columns={'url':'page_count'}, inplace=True)

In [146]:
groupby_domain

,page_count,innovate_count
domain,,
accelerationlab.upenn.edu,1,45
aisp.upenn.edu,2,0
almanac.upenn.edu,3,4
apps.wharton.upenn.edu,2,0
asc.upenn.edu,1,0
...,...,...
wistar.org,1,0
wrds-wharton.upenn.edu,1,21
wsp.wharton.upenn.edu,2,0


site-site matrix

In [147]:
with pd.ExcelWriter('site_domain.xlsx') as writer:  
    links.to_excel(writer, sheet_name='site', index=False)
    groupby_domain.to_excel(writer, sheet_name='domain')

In [166]:
link = links.url[5]
domain_name=[]
print(link)
link_element = link.split('/')
link_element[2] = link_element[2].replace("www.","")
reversed_element  = link_element[2].split('.')[::-1]
print(reversed_element)
reversed_element = ['.'.join(reversed_element[:i][::-1]) for i in range(1,len(reversed_element)+1)]
print(reversed_element)
link_element[2] = '/'.join(reversed_element)
print(len(link_element))
if len(link_element) <= 4:
    domain_name.append(link_element[2])
else:
    for i in range(2, len(link_element)-2):
        if len(link_element[i]) != 0:
            domain_name.append(link_element[i])
html_name = link_element[-2]
# print(link_element)
domain_name = '/'.join(domain_name)
print(domain_name)
# print(link)
# filepath = '/Users/wanxing/PycharmProjects/PennEcosystem/html_data/c3i'
# filepath = os.path.join(destination_dir, domain_name)
# # print(filepath)
# if not os.path.exists(filepath):
#     if not os.path.exists(destination_dir):
#         os.makedirs(destination_dir)
#     os.makedirs(filepath)
# filename = os.path.join(filepath, name+".html")
# urllib.request.urlretrieve(link, filename)

https://executiveeducation.wharton.upenn.edu/for-individuals/all-programs/mastering-innovation-strategy-process-and-tools/
['edu', 'upenn', 'wharton', 'executiveeducation']
['edu', 'upenn.edu', 'wharton.upenn.edu', 'executiveeducation.wharton.upenn.edu']
7
edu/upenn.edu/wharton.upenn.edu/executiveeducation.wharton.upenn.edu/for-individuals/all-programs


In [ ]:

lst= requests.get(url[0], verify=False)
lst= sum(lst.text.count(x) for x in ('innovation', 'innovations', 'innovate'))

In [31]:
import urllib.request
import os

destination_dir = 'home/mgmt/wanxing/html_data/'
# print(link)
for link in links:
    domain_name=[]
    link_element = link.split('/')
    link_element[2] = link_element[2].replace("www.","").replace(".edu","")
    reversed_element  = link_element[2].split('.')[::-1]
    link_element[2] = '/'.join(reversed_element)
    for i in range(2, len(link_element)-2):
        if len(link_element[i]) != 0:
            domain_name.append(link_element[i])
    name = link_element[-2]
    # print(link_element)
    domain_name = '/'.join(domain_name)
    # print(domain)
    # print(link)
    # filepath = '/Users/wanxing/PycharmProjects/PennEcosystem/html_data/c3i'
    filepath = os.path.join(destination_dir, domain_name)
    # print(filepath)
    if not os.path.exists(filepath):
        if not os.path.exists(destination_dir):
            os.makedirs(destination_dir)
        os.makedirs(filepath)
    filename = os.path.join(filepath, name+".html")
    try:
        urllib.request.urlretrieve(link, filename)
    except:
        continue
    # print(filename)
    print("Downloading "+name)
    time.sleep(5)

IndexError: list index out of range

In [3]:
## Google search
import time
import urllib.request
import os

text= "innovation"
domain = "q=site%3Aupenn.edu+"
links = [] # Initiate empty list to capture final results
destination_dir = 'smb://hpcc.wharton.upenn.edu/wanxing/html_data/'
# Specify number of pages on google search, each page contains 10 #links

n_pages = 14200
for page in range(1, n_pages):
    # print(page)
    url = "https://www.google.com/search?" + domain + text + "&start=" + str((page - 1) * 10)
    # print(url)
    driver.get(url)
    soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')
    # print(soup)
    search = soup.find_all('div', class_="yuRUbf")
    # print(search)
    for h in search:
        links.append(h.a.get('href'))
    for h in search:
        domain=[]
        link = h.a.get('href')
        # print(link)
        if link is not None:
            link_element = link.split('/')
            link_element[2] = link_element[2].replace("www.","").replace(".upenn.edu","")
            for i in range(2, len(link_element)-2):
                if len(link_element[i]) != 0:
                    domain.append(link_element[i])
            name = link_element[-2]
            # print(link_element)
            domain = '/'.join(domain)
            title = h.a.text
            links.append(link)
            # print(domain)
            # print(link)
            # filepath = '/Users/wanxing/PycharmProjects/PennEcosystem/html_data/c3i'
            filepath = os.path.join(destination_dir, domain)
            # print(filepath)
            if not os.path.exists(filepath):
                if not os.path.exists(destination_dir):
                    os.makedirs(destination_dir)
                os.makedirs(filepath)
            filename = os.path.join(filepath, name+".html")
            try:
                urllib.request.urlretrieve(link, filename)
            except:
                continue
            # print(filename)
            print("Downloading "+title)
            time.sleep(10)

KeyboardInterrupt: 

In [4]:
links

['https://www.nursing.upenn.edu/innovation/',
 'https://www.nursing.upenn.edu/innovation/innovation-accelerator/',
 'https://knowledge.wharton.upenn.edu/article/business-model-innovation-matters-ever/',
 'https://knowledge.wharton.upenn.edu/article/how-big-companies-can-innovate-like-small-startups/',
 'https://www.med.upenn.edu/evpdean/innovation.html',
 'https://executiveeducation.wharton.upenn.edu/for-individuals/all-programs/mastering-innovation-strategy-process-and-tools/',
 'https://pc3i.upenn.edu/',
 'https://stevenscenter.wharton.upenn.edu/',
 'https://globalyouth.wharton.upenn.edu/programs-courses/essentials-of-innovation/',
 'https://www.hr.upenn.edu/PennHR/penn-community/inclusion-innovation-impact',
 'https://www.nursing.upenn.edu/innovation/',
 'https://www.nursing.upenn.edu/innovation/innovation-accelerator/',
 'https://knowledge.wharton.upenn.edu/article/business-model-innovation-matters-ever/',
 'https://knowledge.wharton.upenn.edu/article/how-big-companies-can-innovate

In [ ]:
import pandas as pd
links = pd.DataFrame(links, columns=['link'])
links.to_csv('links.csv')

In [ ]:
# import urllib.request
# import os
# # Penn Website search https://www.upenn.edu/search
# text= "innovation"
# links = [] # Initiate empty list to capture final results
# destination_dir = '/Users/wanxing/PycharmProjects/PennEcosystem/html_data/'
# # Specify number of pages on google search, each page contains 10 #links

# n_pages = 20
# for page in range(1, n_pages):
#    #https://www.upenn.edu/search?as_q=innovation#gsc.tab=0&gsc.q=innovation&gsc.page=1 
#     url = "https://www.upenn.edu/search?as_q=" + text + "#gsc.tab=0&gsc.q=innovation&gsc.page=" + str((page))
#     driver.get(url)
#     soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')
#     search = soup.find_all('div', class_="gsc-thumbnail-inside")
#     for h in search:
#         domain=[]
#         link = h.a.get('href')
#         # print(link)
#         if link is not None:
#             link_element = link.split('/')
#             for i in range(2, len(link_element)):
#                 if len(link_element[i]) != 0:
#                     domain.append(link_element[i])
#             domain = '/'.join(domain)
#             title = h.a.text
#             links.append(link)
#             print(page)
#             print(domain)
#             print(link)

#             filepath = os.path.join(destination_dir, domain)
#             print(filepath)
#             if not os.path.exists(filepath):
#                 if not os.path.exists(destination_dir):
#                     os.makedirs(destination_dir)
#                 os.makedirs(filepath)
#             try:
#                 urllib.request.urlretrieve(link, filepath+'.html')
#             except:
#                 continue
#             print("Downloading "+title)

In [ ]:
#   # Make two strings with default google search URL
# # 'https://google.com/search?q=' and
# # our customized search keyword.
# # Concatenate them
# text= "innovation"
# domain = "site%3Aupenn.edu+"
# url = 'https://google.com/search?q=' + domain + text
  
# # Fetch the URL data using requests.get(url),
# # store it in a variable, request_result.
# request_result=requests.get( url )
  
# # Creating soup from the fetched request
# soup = bs4.BeautifulSoup(request_result.text,
#                          "html.parser")
# print(soup)

In [ ]:
# # soup.find.all( h3 ) to grab 
# # all major headings of our search result,
# heading_object=soup.find_all( 'h3' )
  
# # Iterate through the object 
# # and print it as a string.
# for info in heading_object:
#     print(info.getText())
#     print("------")